## AI-driven Customer Feedback Analysis and Chatbot Development for Enhancing User Experience

Video Link: https://drive.google.com/file/d/18SW-8XKknv-icJ4j6ZRmK2O5H8D3dQ-R/view

### SCRAPE GOOGLE REVIEW DATA

In [3]:
#The data is collected from google play store for “skiptheDishes” app using google play scraper. 
from google_play_scraper import app, reviews

# Specify the package name of the app for which you want to extract reviews
package_name = 'com.ncconsulting.skipthedishes_android'

# Fetch the reviews for the specified app
result = reviews(package_name, lang='en', count=5000,country="ca",continuation_token=None)  # Specify the number of reviews to fetch

# Print the fetched reviews
#print(len(result[0]))

In [4]:
import numpy as np
import pandas as pd
from transformers import pipeline
import warnings
warnings.filterwarnings("ignore")

reviews_skip=pd.read_csv("data.csv")

### SENTIMENT ANALYSIS

In [5]:
#Sentiment analysis's main goal is to learn more about how consumers feel about the SkipTheDishes app in general. 
#Stakeholders can determine the app's features and services' strong and weak points by analyzing the sentiment distribution among reviews.
#We are categorizing the reviews as sentiment score and sentiment label, indicating the class POSITIVE or NEGATIVE, using hugging face transformers 
#pretrained models. 
import warnings
warnings.filterwarnings("ignore")

summarization_pipeline = pipeline("sentiment-analysis",model="distilbert/distilbert-base-uncased-finetuned-sst-2-english")
reviews_skip['Sentiment_score']=np.nan
reviews_skip['Sentiment_class']=np.nan
from tqdm import tqdm
for j in tqdm(range(0,len(reviews_skip))):
    res=summarization_pipeline(reviews_skip['content'][j])
    reviews_skip['Sentiment_score'][j]=res[0]['score']
    reviews_skip['Sentiment_class'][j]=res[0]['label']

All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.
100%|██████████████████████████████████████████████████████████████████████████████| 1592/1592 [08:09<00:00,  3.25it/s]


### EMOTION ANALYSIS

In [6]:
#Finding the underlying emotions influencing customer feedback is the aim of emotion analysis. 
#Stakeholders can better understand the underlying reasons for customer satisfaction or dissatisfaction and adjust their responses
#by understanding the emotional states of their users.
#We are categorizing the reviews as an emotion score and an emotion label, which indicate the various classes of the emotions, using hugging face
#transformers pretrained models.
import warnings
warnings.filterwarnings("ignore")

pipe = pipeline("text-classification", model="finiteautomata/bertweet-base-emotion-analysis",max_length=512,truncation=True)
reviews_skip['Emotion_score']=np.nan
reviews_skip['Emotion_class']=np.nan
from tqdm import tqdm
for j in tqdm(range(0,len(reviews_skip))):
    try:
        res=pipe(reviews_skip['content'][j])
        reviews_skip['Emotion_score'][j]=res[0]['score']
        reviews_skip['Emotion_class'][j]=res[0]['label']
    except:
        reviews_skip['Emotion_score'][j]=np.nan
        reviews_skip['Emotion_class'][j]=np.nan

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
emoji is not installed, thus not converting emoticons or emojis into text. Instal

### TOPIC MODELLING - LDA

In [7]:
#To find latent topics or themes in the corpus of customer reviews, topic modeling approaches like Latent Dirichlet Allocation (LDA) or
#Non-Negative Matrix Factorization (NMF) are used. These algorithms use the word distribution to automatically classify reviews into subjects. 
#Finding common issues, worries, or areas of interest among clients is the aim of topic modeling. Stakeholders can prioritize and address the most 
#pertinent issues influencing user experience by grouping reviews into topics.

#LDA is a generative probabilistic model. 
#The LDA model generates a distribution of topics, where each topic is represented by a distribution of terms (words) from the corpus. 
#These distributions provide information about which terms are most associated with each topic.
import re
import numpy as np
import pandas as pd
from pprint import pprint

import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess
from gensim.models.ldamodel import LdaModel

import nltk
nltk.download('stopwords')
import warnings
warnings.filterwarnings("ignore")

from nltk.corpus import stopwords
def preprocess_data(documents):
    stop_words = stopwords.words('english')
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in documents]
    return texts


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hemak\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
documents=reviews_skip['content']
processed_texts = preprocess_data(documents)
id2word = corpora.Dictionary(processed_texts)
# Create Corpus
texts = processed_texts
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [9]:
#The code initializes an LDA model with specific parameters such as the number of topics (num_topics), the document-term matrix (corpus) 
#representing term frequencies in each document, and a dictionary (id2word) mapping word IDs to words.
#Coherence is a measure of how interpretable and distinct the topics discovered by the model are.
num_topics = 2
# Build LDA model
lda_model = LdaModel(corpus=corpus, id2word=id2word, num_topics=num_topics, random_state=42, passes=10, alpha='auto', per_word_topics=True)
coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.5185896430672139


In [10]:
pprint(lda_model.print_topics())

[(0,
  '0.024*"food" + 0.022*"service" + 0.021*"order" + 0.017*"app" + 0.016*"skip" '
  '+ 0.015*"delivery" + 0.014*"get" + 0.013*"drivers" + 0.012*"customer" + '
  '0.010*"refund"'),
 (1,
  '0.020*"order" + 0.019*"app" + 0.012*"service" + 0.011*"customer" + '
  '0.010*"skip" + 0.010*"great" + 0.007*"support" + 0.006*"time" + 0.006*"get" '
  '+ 0.005*"even"')]


In [11]:
#PyLDAvis calculates the similarity (or dissimilarity) between topics based on their distributions of terms. 
#After calculating the distances between topics, MDS is applied to reduce the dimensionality of the topic space while preserving the pairwise 
#distances as much as possible. MDS techniques like Principal Component Analysis (PCA) are commonly used for this purpose.
#By using pyLDAvis we are visualizing the topics as 1 and 2 with the word’s relevance to their
#respective topics.
import pyLDAvis.gensim_models as gensimvis
import gensim
from gensim.corpora import Dictionary
import pyLDAvis

# Convert corpus to a bag-of-words representation
dictionary = Dictionary.from_corpus(corpus, lda_model.id2word)
#corpus_bow = [dictionary.doc2bow(doc) for doc in corpus]

# Visualize the LDA model
lda_vis = gensimvis.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(lda_vis)

In [12]:
topic_distribution = [lda_model[doc] for doc in corpus]

# Function to get the dominant topic for each document
def get_dom_topic(topic_dist):
    if not topic_dist:
        return None
    return max(topic_dist[0])[0]
# Assign dominant topics to each document
documents_topics = [get_dom_topic(doc) for doc in topic_distribution]


In [13]:
reviews_skip['Topics']=documents_topics
reviews_skip['Topics_LDA']=reviews_skip['Topics'].apply(lambda x: 'Food Based' if x == 0 else 'App Based')

### TOPIC MODELLING - NMF

In [14]:
#NMF is a dimensionality reduction technique used for feature extraction and topic modeling. 
#It factorizes the TF-IDF matrix into two matrices: W (document-topic matrix) and H (topic-term matrix) to approximate the original TF-IDF matrix.
#Each row of the W matrix represents a document's distribution over topics, and each column of the H matrix represents a topic's distribution over terms.

import plotly.graph_objs as go
from plotly.subplots import make_subplots
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import warnings
warnings.filterwarnings("ignore")
content=reviews_skip['content']
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
stop_words=list(stopwords.words('english'))
stop_words.append("skip")

# Create TF-IDF vectorizer
vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words=stop_words,ngram_range=(2,4))
tfidf = vectorizer.fit_transform(content)

# Apply NMF
num_topics = 4
nmf_model = NMF(n_components=num_topics, random_state=42)
nmf_output = nmf_model.fit_transform(tfidf)

# Visualization - Topic-Word Distribution
feature_names = vectorizer.get_feature_names_out()
for topic_idx, topic in enumerate(nmf_model.components_):
    if topic_idx in range (0,2):
        temp="Customer Experience"
    elif topic_idx==2:
        temp="App Experience"
    else:
        temp="Overall Delivery Experience"
    #print(f"Word Cloud for Topic :{temp}")
    top_words_idx = topic.argsort()[:-20:-1]
    #print(top_words_idx)
    top_words = [feature_names[i] for i in top_words_idx]
    print(temp)
    print(top_words)

import numpy as np
feature_names = vectorizer.get_feature_names_out()
nmf_components = nmf_model.components_
num_top_words = 10

top_words = []
word_weights = []
topic_names = []

for topic_idx, topic in enumerate(nmf_components):
    top_feature_idxs = topic.argsort()[-num_top_words:][::-1]
    top_words.append([feature_names[i] for i in top_feature_idxs])
    word_weights.append(topic[top_feature_idxs])
    topic_names.append(f"Topic {topic_idx + 1}")
Topic=[]
Word=[]
Weight=[]
# Display top words, their weights, and topic names
for topic_idx, (topic_words, weights, topic_name) in enumerate(zip(top_words, word_weights, topic_names)):
    #print(f"Topic {topic_idx + 1}: {topic_name}")
    for word, weight in zip(topic_words, weights):
        #print(f"{word}: {weight}")
        Topic.append(topic_name)
        Word.append(word)
        Weight.append(weight)
nmf=pd.DataFrame()
nmf['Topic']=Topic
nmf['Word']=Word
nmf['Weight']=Weight

#The output of the NMF model which are the topics categorized are visualized using a topic_wizard using dash library.
# Initialize Dash app
app = dash.Dash(__name__)

# Define layout
app.layout = html.Div([
    html.H1('Topic Wizard'),
    dcc.Dropdown(
        id='topic-dropdown',
        options=[
            {'label': 'Customer Experience', 'value': 'customer_experience'},
            {'label': 'App Experience', 'value': 'app_experience'},
            {'label': 'Overall Delivery Experience', 'value': 'overall_delivery_experience'}
        ],
        value='customer_experience'
    ),
    dcc.Graph(id='topic-visualization')
])

@app.callback(
    Output('topic-visualization', 'figure'),  # Output for word cloud
    Input('topic-dropdown', 'value')
)

def update_topic_visualization(selected_topic):
    # Generate visualization based on selected topic
    if selected_topic == 'customer_experience':
        data=nmf[(nmf['Topic']=='Topic 1' ) | (nmf['Topic']=='Topic 2')].sort_values('Weight',ascending =False)[0:10]
        #fig = go.Figure(data=[go.Bar(x=df['Category'], y=df['Value'])],layout={'title': 'Bar Chart'})
        fig = go.Figure(data=[go.Bar(x=data['Word'], y=data['Weight'])])
        fig.update_layout(title='Customer Experience Topics', xaxis_title='Topics', yaxis_title='Weight')

    elif selected_topic == 'app_experience':
        # Similar logic for app experience, assuming you have topic data for app experience
        data=nmf[(nmf['Topic']=='Topic 3')].sort_values('Weight',ascending =False)
        #fig = go.Figure(data=[go.Bar(x=df['Category'], y=df['Value'])],layout={'title': 'Bar Chart'})
        fig = go.Figure(data=[go.Bar(x=data['Word'], y=data['Weight'])])
        fig.update_layout(title='App Experience Topics', xaxis_title='Topics', yaxis_title='Weight')

    elif selected_topic == 'overall_delivery_experience':
        data=nmf[(nmf['Topic']=='Topic 4' )].sort_values('Weight',ascending =False)
        #fig = go.Figure(data=[go.Bar(x=df['Category'], y=df['Value'])],layout={'title': 'Bar Chart'})
        fig = go.Figure(data=[go.Bar(x=data['Word'], y=data['Weight'])])
        fig.update_layout(title='Overall Delivery Topics', xaxis_title='Topics', yaxis_title='Weight')


    else:
        # Handle case where selected topic does not match any of the predefined options
        fig = go.Figure()
    fig.update_layout(xaxis=dict(
        automargin=True,  # Automatically adjust margin to prevent overlap
        tickmode='array',  # Set tick mode to array
        tickvals=data['Word'],  # Set tick values to the data
        ticktext=data['Word'].apply(lambda x: '<br>'.join(x.split())),  # Split text and join with <br> for new line
    ))
    return fig


# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


Customer Experience
['great service', 'far good', 'service always', 'service time', 'service everytime', 'great food', 'great service quick', 'service quick', 'always great service', 'great service great', 'always great', 'service great', 'easy convenient', 'food great', 'food great service', 'please keep', 'go beyond', 'quick easy', 'points last']
Customer Experience
['customer service', 'poor customer', 'poor customer service', 'worst customer service', 'worst customer', 'horrible customer service', 'horrible customer', 'worst customer service ever', 'customer service ever', 'service ever', 'app customer', 'app customer service', 'terrible customer service', 'terrible customer', 'every time', 'customer service sucks', 'service sucks', 'good customer', 'good customer service']
App Experience
['easy use', 'good service', 'convenient easy', 'convenient easy use', 'quick easy', 'good easy', 'highly recommended', 'needs work', 'great app', 'lots options', 'app easy', 'good selection', 'us

In [15]:
# Update Topics to main dataframe
topic_num=[]
dominant_topics = np.argmax(nmf_output, axis=1)
for idx, document in enumerate(content):
    if dominant_topics[idx] in range (0,2):
        topic_num.append("Customer Experience")
    elif dominant_topics[idx]==2:
        topic_num.append("App Experince")
    else:
        topic_num.append("Overall Delivery Experince")
reviews_skip['Topics_NMF']=topic_num


#### COMBINED RESULT

In [16]:
reviews_skip=reviews_skip.drop(['replyContent','repliedAt','score','thumbsUpCount','reviewCreatedVersion','at','appVersion','Topics'],axis=1)

In [17]:
reviews_skip.head(5)

,reviewId,userName,userImage,content,Sentiment_score,Sentiment_class,Emotion_score,Emotion_class,Topics_LDA,Topics_NMF
0,6a46db96-cb7e-40c0-8790-694df2f12a46,Dakota Leveille,https://play-lh.googleusercontent.com/a-/ALV-U...,Delivered my food peferectly to my described l...,0.999785,POSITIVE,0.977111,joy,App Based,Overall Delivery Experince
1,9c7634f8-e224-4634-bcba-550fd6176871,Scott Davidson,https://play-lh.googleusercontent.com/a-/ALV-U...,Customer service is lacking. It's really gone ...,0.999805,NEGATIVE,0.961397,disgust,App Based,Customer Experience
2,616c2389-7b74-467c-bfac-2486ec434631,Tanis Nicholson,https://play-lh.googleusercontent.com/a/ACg8oc...,additional fees are getting ridiculous,0.999638,NEGATIVE,0.947310,disgust,App Based,Overall Delivery Experince
3,98a573af-b8e6-4105-a605-2863009fdf67,Colin Carroll,https://play-lh.googleusercontent.com/a-/ALV-U...,The absolute worst customer service. Drivers d...,0.999781,NEGATIVE,0.969821,disgust,App Based,Customer Experience
4,440de9bc-f4eb-46af-be19-36fde3a91703,JL Rae,https://play-lh.googleusercontent.com/a/ACg8oc...,extra fees keep adding up,0.995747,POSITIVE,0.925498,others,App Based,Customer Experience


### CHATBOT DEVELOPMENT AND INTERFACE

In [18]:

import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from gensim.test.utils import common_texts
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from transformers import pipeline
import warnings
warnings.filterwarnings("ignore")
from sumy.parsers.plaintext import PlaintextParser                   
from sumy.nlp.tokenizers import Tokenizer                      
from sumy.summarizers.text_rank import TextRankSummarizer 

df=reviews_skip
def preprocess(text):
    # Tokenize
    tokens = word_tokenize(text.lower())
    # Remove punctuation and stop words
    tokens = [word for word in tokens if word not in stopwords.words("english") and word not in string.punctuation]
    return tokens

#This function accepts a user question as input and utilizes pre-trained models to predict the emotion and sentiment associated with the question. 
def response(question):
    emotion_p=pipe(question)[0]['label']
    sentiment_p=summarization_pipeline(question)[0]['label']
    filtered_df = df[(df['Sentiment_class'] == sentiment_p) & (df['Emotion_class'] == emotion_p)]
    documents=filtered_df['content'].tolist()
    queries = [question]
    preprocessed_documents = [preprocess(doc) for doc in documents]
    preprocessed_queries = [preprocess(query) for query in queries]
    
#It then filters a reviews based on predicted sentiment and emotion, retrieves content from the filtered DataFrame as documents,
#tokenizes and preprocesses the documents, and trains a Word2Vec model on preprocessed documents to obtain word embeddings. 
#Cosine similarity is computed between the question and each document to select top relevant documents based on similarity scores.
#Finally, the TextRank summarization algorithm is applied to summarize the selected documents, and the summarized text is returned.    
    
    model = Word2Vec(sentences=preprocessed_documents, vector_size=100, window=5, min_count=1, workers=4,epochs=100)
    for i, query in enumerate(preprocessed_queries):
        query_vector = np.zeros((model.vector_size,))
        for word in query:
            if word in model.wv:
                query_vector += model.wv[word]
        query_vector /= len(query)  # Normalize query vector
        relevant_documents = []
        for j, document in enumerate(preprocessed_documents):
            document_vector = np.zeros((model.vector_size,))
            for word in document:
                if word in model.wv:
                    document_vector += model.wv[word]
            document_vector /= len(document)  # Normalize document vector
            similarity = np.dot(query_vector, document_vector) / (np.linalg.norm(query_vector) * np.linalg.norm(document_vector))
            relevant_documents.append((documents[j], similarity))
    relevant_documents.sort(key=lambda x: x[1], reverse=True)
    print(f"Query: {queries[i]}")
    top_10_documents = relevant_documents[:30]
    doc_main=[]
    for doc, similarity in top_10_documents:
        if similarity > 0.3 :
            doc_main.append(doc)
    parser = PlaintextParser.from_string('.'.join(doc_main),Tokenizer("english"))         
    # Summarize using sumy TextRank                  
    summarizer_4 = TextRankSummarizer()                   
    summary =summarizer_4(parser.document,4)                   
    text_summary=""                  
    for sentence in summary:                
        text_summary+=str(sentence)   
    #print(doc_main)

    return text_summary        


In [19]:
#It sets up a Gradio interface (iface) by specifying the function to be used (chatbot_interface), the input type (text), and the output type (text).
#It also provides a title and description for the interface.

import gradio as gr
import matplotlib.pyplot as plt
#import openAI
def chatbot_interface(input_text):
    # Use GPT-3 to generate a response
    response_1 = response(input_text)
    stop_words = set(stopwords.words('english'))
    words = response_1.split()
    filtered_text = [word for word in words if word.lower() not in stop_words]
    wordcloud = WordCloud(width=800, height=400, background_color ='white').generate(' '.join(filtered_text))    
    # Display the word cloud
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title("Word Cloud")
    plt.savefig("wcplot.png")
    return response_1.strip(),"wcplot.png"


input_textbox = gr.Textbox(label="Enter your question here")
output_textbox = gr.Textbox(label="Chatbot Response")
output_image = gr.Image(label="Word Cloud")
# Define Gradio interface
iface = gr.Interface(
    fn=chatbot_interface,
    inputs=input_textbox,
    outputs=[output_textbox,output_image],
    title="Review QA",
    description="Enter a Question and the chatbot will respond based upon customer reviews"
)

# Launch the interface
iface.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


## References

#### Gradio
Abid, A. (n.d.). Interface. Gradio. \
https://www.gradio.app/docs/interface
#### Hugging face- Emotion Prediction
Finite Automata. (n.d.). BERTweet-base-emotion-analysis. \
https://huggingface.co/finiteautomata/bertweet-base-emotion-analysis
#### Hugging face- Sentiment Prediction
Hugging Face. (n.d.). DistilBERT-base-uncased-finetuned-sst-2-english. \
https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english
#### Google Play Scraper
JoMingyu. (n.d.). Google-play-scraper. GitHub. \
https://github.com/JoMingyu/google-play-scraper.
#### Dash Plotly
Plotly Technologies Inc. (n.d.). Dash User Guide. Plotly. \
https://dash.plotly.com/#dash-fundamentals
#### LDA
Rehurek, R. (n.d.). Topic modeling for fun and profit. Gensim. \
https://radimrehurek.com/gensim/auto_examples/tutorials/run_lda.html
#### Word2Vec
Radim Rehurek. (n.d.). Word2Vec - Gensim. \
https://radimrehurek.com/gensim/models/word2vec.html.
#### NMF Model
Scikit-learn developers. (n.d.). sklearn.decomposition.NMF. Scikit-learn: Machine Learning in Python. \
https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html
#### LDAvis
Sievert, C., & Shirley, K. E. (n.d.). LDAvis: A method for visualizing and interpreting topics. \
https://nlp.stanford.edu/events/illvi2014/papers/sievert-illvi2014.pdf
http://vis.stanford.edu/files/2012-Termite-AVI.pdf.

